In [1]:
import numpy as np
import pandas as pd

In [4]:
X = np.load('/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/data/X.npy',allow_pickle= True)
y = np.load('/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/data/y.npy',allow_pickle= True)

In [5]:
X.shape

(41157,)

In [8]:
from Modules.gensim_vectorizers import FastText_vectorize
vec = FastText_vectorize("/media/senju/1.0 TB Hard Disk/NLP/FastText_Classification/fasttext/wiki-news-300d-1M.vec")

In [9]:
vec._build_vector()

999995it [01:01, 16196.71it/s]


In [10]:
vec._get_vector_list(X[0]).shape

(12, 300)

In [11]:
X, y = X[:1000] , y[:1000]

In [12]:
print(X.shape, y.shape)

(1000,) (1000,)


In [13]:
pd.DataFrame(y).value_counts()

0    429
2    405
1    166
dtype: int64

In [14]:
vec_list = [vec.padding_truncate(vec._get_vector_list(X[i]),max_length=20) for i in range(X.shape[0])]

In [15]:
# np.array(vec_list).shape

In [16]:
np.array(vec_list).shape

(1000, 300, 20)